# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json
import time

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv", index_col="City_ID")

# Display sample data
city_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
City_ID,,,,,,,,,
0,laguna,38.4210,-121.4238,17.48,68,0,0.45,US,2024-09-13
1,bethel,41.3712,-73.4140,23.97,79,6,1.15,US,2024-09-13
2,isafjordur,66.0755,-23.1240,7.02,61,0,1.54,IS,2024-09-13
3,mancio lima,-7.6142,-72.8958,30.04,70,12,1.54,BR,2024-09-13
4,warmbad,-28.4500,18.7333,26.12,9,2,4.16,NaN,2024-09-13


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
plot_1 = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True, 
    tiles = 'OSM', 
    frame_width = 700, 
    frame_height = 500, 
    color='City',
    alpha= 0.5,
    s='Humidity',
    scale= 0.8)

# Display the map
plot_1


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [9]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_df = city_data_df[
    (city_data_df['Max Temp'] > 21) & (city_data_df['Max Temp'] < 27) &
    (city_data_df['Wind Speed'] < 4.5) & (city_data_df['Cloudiness'] == 0)
]
print(len(ideal_df))

# Drop any rows with null values
ideal_df.dropna()

# Display sample data
ideal_df.head()

14


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
City_ID,,,,,,,,,
37,albany,42.6001,-73.9662,22.67,75,0,0.45,US,2024-09-13
71,saint-felix-de-valois,46.1668,-73.4324,24.58,68,0,0.45,CA,2024-09-13
143,kill devil hills,36.0307,-75.6760,24.99,85,0,3.09,US,2024-09-13
168,jamestown,42.0970,-79.2353,22.79,61,0,2.57,US,2024-09-13
216,bay city,43.5945,-83.8889,23.92,68,0,2.57,US,2024-09-13


### Step 3: Create a new DataFrame called `hotel_df`.

In [15]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_df.iloc[:,[0,7,1,2,4]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name']=''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
City_ID,,,,,,
37,albany,US,42.6001,-73.9662,75,
71,saint-felix-de-valois,CA,46.1668,-73.4324,68,
143,kill devil hills,US,36.0307,-75.6760,85,
168,jamestown,US,42.0970,-79.2353,61,
216,bay city,US,43.5945,-83.8889,68,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [17]:
# Set parameters to search for a hotel
radius = 10000
params = {'categories':'accommodation.hotel',\
         'filter':'',\
         'bias':'',\
         'apiKey':geoapify_key,\
         'limit':1}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lon = row['Lng']
    lat = row['Lat']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lon},{lat},{radius}'
    params["bias"] = f'proximity:{lon},{lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
#         # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
#     # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# # Display sample data
hotel_df.head()

Starting hotel search
albany - nearest hotel: No hotel found
saint-felix-de-valois - nearest hotel: L'Auberge de la Montagne Coupée
kill devil hills - nearest hotel: Mariner Days Inn & Suites
jamestown - nearest hotel: DoubleTree Jamestown
bay city - nearest hotel: Comfort Inn
qiongshan - nearest hotel: 全季酒店
westport - nearest hotel: Norwalk Inn and Conference Center
ankang - nearest hotel: 金友商务酒店
haiku-pauwela - nearest hotel: Inn At Mama's Fish House
santa rosa de rio primero - nearest hotel: Hotel presidente
moapa valley - nearest hotel: No hotel found
tremp - nearest hotel: Hotel Segle XX
kandahar - nearest hotel: No hotel found
mentor-on-the-lake - nearest hotel: Best Western


,City,Country,Lat,Lng,Humidity,Hotel Name
City_ID,,,,,,
37,albany,US,42.6001,-73.9662,75,No hotel found
71,saint-felix-de-valois,CA,46.1668,-73.4324,68,L'Auberge de la Montagne Coupée
143,kill devil hills,US,36.0307,-75.6760,85,Mariner Days Inn & Suites
168,jamestown,US,42.0970,-79.2353,61,DoubleTree Jamestown
216,bay city,US,43.5945,-83.8889,68,Comfort Inn


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [20]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE


# Configure the map plot
plot_4 = hotel_df.hvplot.points('Lng',
            'Lat',
            geo=True, 
            tiles = 'OSM', 
            frame_width = 700, 
            frame_height = 500, 
            color='City',
            alpha=0.6,
            hover=True,
            hover_cols='all',
            use_index=False,
            s='Humidity',
            scale= 1.2)

# Display the map
plot_4


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)